## Evaluating Elasticsearch

In [46]:
import json


from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

In [2]:
with open('notebooks/documents-with-ids.json', 'r') as file:
    documents = json.load(file)

documents[10]

{'Category': 'Temporarily unable to work',
 'Question': "Can I get sick pay if I'm self-isolating?",
 'Answer': "Yes Statutory Sick Pay is available if you're self-isolating.",
 'Section': 'general claim benefits',
 'id': '1de35e0b-f233-554c-84ef-fc30494e0ea0'}

In [3]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "Answer": {"type": "text"},
            "Category": {"type": "text"},
            "Question": {"type": "text"},
            "Section": {"type": "keyword"},
            "id": {"type": "keyword"},
        }
    }
}

index_name = "benefit-claims"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'benefit-claims'})

In [4]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/425 [00:00<?, ?it/s]

In [5]:
def elastic_search(query, section):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["Question^3", "Answer", "Category"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "Section": section
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [6]:
elastic_search(
    query="Can I get sick pay if I'm self-isolating?",
    section="general claim benefits"
)

[{'Category': 'Temporarily unable to work',
  'Question': "Can I get sick pay if I'm self-isolating?",
  'Answer': "Yes Statutory Sick Pay is available if you're self-isolating.",
  'Section': 'general claim benefits',
  'id': '1de35e0b-f233-554c-84ef-fc30494e0ea0'},
 {'Category': 'Temporarily unable to work',
  'Question': 'How do I apply for sick pay?',
  'Answer': 'You need to provide a fit note from your doctor to apply for sick pay.',
  'Section': 'general claim benefits',
  'id': 'ecc42084-d88e-5bf0-8070-a18552c283bb'},
 {'Category': 'Temporarily unable to work',
  'Question': 'What is statutory sick pay?',
  'Answer': "Statutory Sick Pay is a legal requirement for employers to pay you if you're ill.",
  'Section': 'general claim benefits',
  'id': 'ee6a77ec-d4ef-50f7-9762-04cc228b3a48'},
 {'Category': 'Disabled or health condition',
  'Question': 'Can I get help with housing if I’m disabled?',
  'Answer': 'You may be eligible for a Disabled Facilities Grant to adapt your home to

In [47]:
import pandas as pd

In [7]:
df_ground_truth = pd.read_csv('notebooks/ground-truth-data.csv')

In [8]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [9]:
ground_truth[10]

{'question': 'Is it possible to appeal?',
 'claims_type': 'general claim benefits',
 'document': '8d000ade-6c2b-571c-aa61-5d38eb463cf8'}

In [10]:
elastic_search(
    query="Can I get sick pay if I'm self-isolating?",
    section="general claim benefits"
)

[{'Category': 'Temporarily unable to work',
  'Question': "Can I get sick pay if I'm self-isolating?",
  'Answer': "Yes Statutory Sick Pay is available if you're self-isolating.",
  'Section': 'general claim benefits',
  'id': '1de35e0b-f233-554c-84ef-fc30494e0ea0'},
 {'Category': 'Temporarily unable to work',
  'Question': 'How do I apply for sick pay?',
  'Answer': 'You need to provide a fit note from your doctor to apply for sick pay.',
  'Section': 'general claim benefits',
  'id': 'ecc42084-d88e-5bf0-8070-a18552c283bb'},
 {'Category': 'Temporarily unable to work',
  'Question': 'What is statutory sick pay?',
  'Answer': "Statutory Sick Pay is a legal requirement for employers to pay you if you're ill.",
  'Section': 'general claim benefits',
  'id': 'ee6a77ec-d4ef-50f7-9762-04cc228b3a48'},
 {'Category': 'Disabled or health condition',
  'Question': 'Can I get help with housing if I’m disabled?',
  'Answer': 'You may be eligible for a Disabled Facilities Grant to adapt your home to

In [11]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = elastic_search(query=q['question'], section=q['claims_type'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/2055 [00:00<?, ?it/s]

In [12]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [13]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [14]:
hit_rate(relevance_total), mrr(relevance_total)

(0.7732360097323601, 0.6168369829683708)

In [15]:
documents[10]

{'Category': 'Temporarily unable to work',
 'Question': "Can I get sick pay if I'm self-isolating?",
 'Answer': "Yes Statutory Sick Pay is available if you're self-isolating.",
 'Section': 'general claim benefits',
 'id': '1de35e0b-f233-554c-84ef-fc30494e0ea0'}

## Minsearch

In [16]:
import minsearch

index = minsearch.Index(
    text_fields=["Question", "Answer", "Category"],
    keyword_fields=["Section", "id"]
)

index.fit(documents)

In [37]:
def min_search(query, section):
    boost = {'Question': 3.0, 'Category': 0.5}

    results = index.search(
        query=query,
        filter_dict={'Section': section},
        boost_dict=boost,
        num_results=5
    )

    return results

In [38]:
ground_truth[0]

{'question': 'How can I change my existing benefit details?',
 'claims_type': 'general claim benefits',
 'document': '30eada08-5708-5c5c-9df8-0f7d5d4dc131'}

In [39]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = min_search(query=q['question'], section=q['claims_type'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/2055 [00:00<?, ?it/s]

In [40]:
hit_rate(relevance_total), mrr(relevance_total)

(0.8306569343065694, 0.6697891321978915)

`ES- Hitrate:0.7732360097323601, MRR:0.6168369829683708)`

In [41]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [43]:
evaluate(ground_truth, lambda q: elastic_search(q['question'], q['claims_type']))

  0%|          | 0/2055 [00:00<?, ?it/s]

{'hit_rate': 0.7732360097323601, 'mrr': 0.616999188969993}

In [45]:
evaluate(ground_truth, lambda q: min_search(q['question'], q['claims_type']))

  0%|          | 0/2055 [00:00<?, ?it/s]

{'hit_rate': 0.8306569343065694, 'mrr': 0.6697891321978915}

## Vector Search

In [48]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

/home/codespace/.local/share/virtualenvs/uk-benefits-assistant-nL9c5Gpa/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [49]:
v = model.encode("Can I get sick pay if I'm self-isolating?")

In [50]:
len(v)

384

In [85]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "Answer": {"type": "text"},
            "category": {"type": "text"},
            "Question": {"type": "text"},
            "Section": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "answer_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_answer_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "benefit-claims"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'benefit-claims'})

In [86]:
documents[0]

{'Category': 'Manage existing benefit',
 'Question': 'How do I update my benefit information?',
 'Answer': 'You can update your benefit information online through your account.',
 'Section': 'general claim benefits',
 'id': '30eada08-5708-5c5c-9df8-0f7d5d4dc131',
 'question_vector': array([-1.80060193e-02,  5.96722253e-02,  1.26943989e-02, -3.90117168e-02,
         2.84865871e-02,  9.22173411e-02,  2.27035414e-02,  2.67567299e-02,
        -3.86952274e-02, -2.83248979e-03, -2.74528917e-02,  2.94101797e-03,
        -3.46847787e-03, -1.18925475e-01,  1.39166079e-02,  3.98957506e-02,
        -1.79248862e-02,  7.15748966e-02, -4.80940826e-02, -3.22361998e-02,
        -1.01363212e-01, -2.29153130e-02, -4.96070758e-02,  4.44727466e-02,
         2.36339159e-02,  6.05530553e-02, -2.41196901e-02,  3.91690135e-02,
        -5.07083908e-03, -1.72626209e-02,  5.26820458e-02, -4.72456887e-02,
         1.85332298e-02, -2.26278715e-02,  3.78295816e-02, -8.19180743e-04,
        -8.93227831e-02,  2.79583

In [87]:
for doc in tqdm(documents):
    question = doc['Question']
    answer = doc['Answer']
    qt = question + ' ' + answer

    doc['question_vector'] = model.encode(question)
    doc['answer_vector'] = model.encode(answer)
    doc['question_answer_vector'] = model.encode(qt)

  0%|          | 0/425 [00:00<?, ?it/s]

In [88]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/425 [00:00<?, ?it/s]

In [89]:
query = "Can I get sick pay if I'm self-isolating?"
v_q = model.encode(query)

In [90]:
def elastic_search_knn(field, vector, section):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "Section": section
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["Answer", "Section", "Question", "Category", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [97]:
elastic_search_knn('question_vector', v_q, 'general claim benefits')

[{'Answer': "Yes Statutory Sick Pay is available if you're self-isolating.",
  'Category': 'Temporarily unable to work',
  'Question': "Can I get sick pay if I'm self-isolating?",
  'id': '1de35e0b-f233-554c-84ef-fc30494e0ea0',
  'Section': 'general claim benefits'},
 {'Answer': 'You need to provide a fit note from your doctor to apply for sick pay.',
  'Category': 'Temporarily unable to work',
  'Question': 'How do I apply for sick pay?',
  'id': 'ecc42084-d88e-5bf0-8070-a18552c283bb',
  'Section': 'general claim benefits'},
 {'Answer': "Statutory Sick Pay is a legal requirement for employers to pay you if you're ill.",
  'Category': 'Temporarily unable to work',
  'Question': 'What is statutory sick pay?',
  'id': 'ee6a77ec-d4ef-50f7-9762-04cc228b3a48',
  'Section': 'general claim benefits'},
 {'Answer': 'You may be eligible for certain benefits even if you’re working depending on your income and circumstances.',
  'Category': 'Manage existing benefit',
  'Question': 'Can I receive b

In [98]:
ground_truth[10]

{'question': 'Is it possible to appeal?',
 'claims_type': 'general claim benefits',
 'document': '8d000ade-6c2b-571c-aa61-5d38eb463cf8'}

In [99]:
def question_vector_knn(q):
    question = q['question']
    section = q['claims_type']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, section)

In [100]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [101]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [102]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [103]:
evaluate(ground_truth, question_vector_knn)

  0%|          | 0/2055 [00:00<?, ?it/s]

{'hit_rate': 0.8637469586374696, 'mrr': 0.7441443633414435}

Compared to Minsearch: {'hit_rate': 0.8306569343065694, 'mrr': 0.6697891321978915}

Compared to Elasticsearch Text only: {'hit_rate': 0.7732360097323601, 'mrr':0.6168369829683708}

In [106]:
def answer_vector_knn(q):
    question = q['question']
    section = q['claims_type']

    v_q = model.encode(question)

    return elastic_search_knn('answer_vector', v_q, section)
evaluate(ground_truth, answer_vector_knn)

  0%|          | 0/2055 [00:00<?, ?it/s]

{'hit_rate': 0.8272506082725061, 'mrr': 0.6879399837793996}

In [107]:
def question_answer_vector_knn(q):
    question = q['question']
    section = q['claims_type']

    v_q = model.encode(question)

    return elastic_search_knn('question_answer_vector', v_q, section)

evaluate(ground_truth, question_answer_vector_knn)

  0%|          | 0/2055 [00:00<?, ?it/s]

{'hit_rate': 0.9304136253041363, 'mrr': 0.8123276561232763}

In [109]:
def elastic_search_knn_combined(vector, section):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "script_score": {
                            "query": {
                                "term": {
                                    "Section": section
                                }
                            },
                            "script": {
                                "source": """
                                    cosineSimilarity(params.query_vector, 'question_vector') + 
                                    cosineSimilarity(params.query_vector, 'answer_vector') + 
                                    cosineSimilarity(params.query_vector, 'question_answer_vector') + 
                                    1
                                """,
                                "params": {
                                    "query_vector": vector
                                }
                            }
                        }
                    }
                ],
                "filter": {
                    "term": {
                        "Section": section
                    }
                }
            }
        },
        "_source": ["Answer", "Section", "Question", "Category", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [110]:
def vector_combined_knn(q):
    question = q['question']
    section = q['claims_type']

    v_q = model.encode(question)

    return elastic_search_knn_combined(v_q, section)

evaluate(ground_truth, vector_combined_knn)

  0%|          | 0/2055 [00:00<?, ?it/s]

{'hit_rate': 0.9211678832116789, 'mrr': 0.8021573398215729}

question_answer_vector : {'hit_rate': 0.9304136253041363, 'mrr': 0.8123276561232763}